<a href="https://colab.research.google.com/github/keerthanachan/MapUp-Data-Analyst/blob/main/Copy_of_python_task_2_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

dataset_3 = pd.read_csv("dataset-3.csv")

dataset_3.head()

,id_start,id_end,distance
0,1001400,1001402,9.7
1,1001402,1001404,20.2
2,1001404,1001406,16.0
3,1001406,1001408,21.7
4,1001408,1001410,11.1


In [ ]:
# TASK - 1

def calculate_distance_matrix(file_path='dataset-3.csv'):

    df = pd.read_csv(file_path)

    # a pivot table to represent the distance matrix
    distance_matrix = df.pivot_table(index='id_start', columns='id_end', values='distance', fill_value=0)

    # Ensure the matrix is symmetric
    distance_matrix = distance_matrix + distance_matrix.T - distance_matrix * (distance_matrix.T != 0)

    # Set diagonal values to 0
    distance_matrix.values[[range(len(distance_matrix))]*2] = 0

    return distance_matrix

# Example usage
distance_matrix = calculate_distance_matrix()
print(distance_matrix.head())

         1001400 1001402 1001404 1001406 1001408 1001410 1001412 1001414  \
1001400      NaN     NaN     NaN     NaN     NaN     NaN     NaN     NaN   
1001402      NaN     0.0    20.2     0.0     0.0     0.0     0.0     0.0   
1001404      NaN    20.2     0.0    16.0     0.0     0.0     0.0     0.0   
1001406      NaN     0.0    16.0     0.0    21.7     0.0     0.0     0.0   
1001408      NaN     0.0     0.0    21.7     0.0    11.1     0.0     0.0   

        1001416 1001418  ... 1001462 1001464 1001466 1001468 1001470 1001472  \
1001400     NaN     NaN  ...     NaN     NaN     NaN     NaN     NaN     NaN   
1001402     0.0     0.0  ...     0.0     0.0     0.0     0.0     0.0     NaN   
1001404     0.0     0.0  ...     0.0     0.0     0.0     0.0     0.0     NaN   
1001406     0.0     0.0  ...     0.0     0.0     0.0     0.0     0.0     NaN   
1001408     0.0     0.0  ...     0.0     0.0     0.0     0.0     0.0     NaN   

        1001488 1004354 1004355  1004356  
1001400     NaN    

In [ ]:
# TASK - 2

def unroll_distance_matrix(distance_matrix):
    # Check if 'id_start' is an index, and reset if needed
    if 'id_start' not in distance_matrix.columns:
        distance_matrix_reset = distance_matrix.reset_index()
    else:
        distance_matrix_reset = distance_matrix.copy()

    # Melt the DataFrame to unroll the distance matrix
    unrolled_df = pd.melt(distance_matrix_reset, id_vars=distance_matrix_reset.columns[0], var_name='id_end', value_name='distance')

    # Filter out rows where 'id_start' is equal to 'id_end'
    unrolled_df = unrolled_df[unrolled_df[distance_matrix_reset.columns[0]] != unrolled_df['id_end']]

    # Sort the DataFrame by 'id_start', 'id_end'
    unrolled_df = unrolled_df.sort_values([distance_matrix_reset.columns[0], 'id_end']).reset_index(drop=True)

    return unrolled_df


distance_matrix = calculate_distance_matrix()
unrolled_distance_df = unroll_distance_matrix(distance_matrix)

# Display the resulting DataFrame
print("Unrolled Distance DataFrame:")
print(unrolled_distance_df)

Unrolled Distance DataFrame:
        index   id_end distance
0     1001400  1001402      NaN
1     1001400  1001404      NaN
2     1001400  1001406      NaN
3     1001400  1001408      NaN
4     1001400  1001410      NaN
...       ...      ...      ...
1801  1004356  1001470      0.0
1802  1004356  1001472      NaN
1803  1004356  1001488      4.0
1804  1004356  1004354      2.0
1805  1004356  1004355      0.0

[1806 rows x 3 columns]


In [ ]:
# TASK - 3


def find_ids_within_ten_percentage_threshold(unrolled_distance_df, reference_value):
    # Filter rows based on the reference value
    reference_rows = unrolled_distance_df[unrolled_distance_df['index'] == reference_value]

    # Calculate the average distance for the reference value
    average_distance = reference_rows['distance'].mean()

    # Calculate the 10% threshold values
    lower_threshold = average_distance - (average_distance * 0.1)
    upper_threshold = average_distance + (average_distance * 0.1)

    # Filter rows within the 10% threshold
    within_threshold = unrolled_distance_df[
        (unrolled_distance_df['distance'] >= lower_threshold) & (unrolled_distance_df['distance'] <= upper_threshold)
    ]

    # Get unique values from id_start column
    result_ids = within_threshold['index'].unique()

    # Sort the result and return
    result_ids.sort()
    return result_ids

# Example usage
# Assuming 'unrolled_distance_df' is the DataFrame from Question 2
# Replace this with your actual DataFrame
# Also, replace 'reference_value' with the actual reference value
reference_value = 1001402
result_ids = find_ids_within_ten_percentage_threshold(unrolled_distance_df, reference_value)

# Display the result
print(f"IDs within 10% threshold of the average distance for {reference_value}:")
print(result_ids)

IDs within 10% threshold of the average distance for 1001402:
[]


In [ ]:
# TASK - 4

def calculate_toll_rate(df):
    # Define rate coefficients for each vehicle type
    rate_coefficients = {
        'moto': 0.8,
        'car': 1.2,
        'rv': 1.5,
        'bus': 2.2,
        'truck': 3.6
    }
    df['distance'].fillna(0, inplace=True)

    # Create new columns for each vehicle type
    for vehicle_type, rate in rate_coefficients.items():
        df[vehicle_type] = rate * df['distance']

    return df

In [ ]:
df_unrolled_distance_with_toll = calculate_toll_rate(unrolled_distance_df)

# Print the resulting DataFrame
print(df_unrolled_distance_with_toll[['index', 'id_end', 'distance', 'moto', 'car', 'rv', 'bus', 'truck']])

        index   id_end  distance  moto  car   rv  bus  truck
0     1001400  1001402       0.0   0.0  0.0  0.0  0.0    0.0
1     1001400  1001404       0.0   0.0  0.0  0.0  0.0    0.0
2     1001400  1001406       0.0   0.0  0.0  0.0  0.0    0.0
3     1001400  1001408       0.0   0.0  0.0  0.0  0.0    0.0
4     1001400  1001410       0.0   0.0  0.0  0.0  0.0    0.0
...       ...      ...       ...   ...  ...  ...  ...    ...
1801  1004356  1001470       0.0   0.0  0.0  0.0  0.0    0.0
1802  1004356  1001472       0.0   0.0  0.0  0.0  0.0    0.0
1803  1004356  1001488       4.0   3.2  4.8  6.0  8.8   14.4
1804  1004356  1004354       2.0   1.6  2.4  3.0  4.4    7.2
1805  1004356  1004355       0.0   0.0  0.0  0.0  0.0    0.0

[1806 rows x 8 columns]


In [ ]:
import pandas as pd
from datetime import datetime, time, timedelta

def calculate_time_based_toll_rates(df):
    # Define time ranges and discount factors
    weekday_time_ranges = [
        (time(0, 0, 0), time(10, 0, 0), 0.8),
        (time(10, 0, 0), time(18, 0, 0), 1.2),
        (time(18, 0, 0), time(23, 59, 59), 0.8)
    ]

    weekend_discount_factor = 0.7

    # Create new columns for start_day, start_time, end_day, and end_time
    df['start_day'] = df['end_day'] = df['start_time'] = df['end_time'] = None

    # Iterate over each unique (id_start, id_end) pair
    for index, row in df.iterrows():
        # Extract unique pair
        id_start, id_end = row['index'], row['id_end']

        # Initialize lists to store time intervals and discount factors
        start_times = []
        end_times = []
        discount_factors = []

        # Iterate over each day of the week
        for i in range(7):
            # Determine weekday or weekend
            if i < 5:  # Weekday
                for start, end, factor in weekday_time_ranges:
                    start_times.append(datetime.combine(datetime(2023, 1, 1), start) + timedelta(days=i))
                    end_times.append(datetime.combine(datetime(2023, 1, 1), end) + timedelta(days=i))
                    discount_factors.append(factor)
            else:  # Weekend
                start_times.append(datetime.combine(datetime(2023, 1, 1), time(0, 0, 0)) + timedelta(days=i))
                end_times.append(datetime.combine(datetime(2023, 1, 1), time(23, 59, 59)) + timedelta(days=i))
                discount_factors.append(weekend_discount_factor)

        # Check if lists have the same length
        assert len(start_times) == len(end_times) == len(discount_factors), "Lists must have equal length"

        # Update DataFrame with time intervals and discount factors
        df.at[index, 'start_day'] = start_times[0].strftime('%A')  # Use the first day's name
        df.at[index, 'end_day'] = end_times[-1].strftime('%A')  # Use the last day's name
        df.at[index, 'start_time'] = start_times
        df.at[index, 'end_time'] = end_times

        # Apply discount factor to vehicle columns
        for vehicle_type in ['moto', 'car', 'rv', 'bus', 'truck']:
            df.at[index, vehicle_type] = row[vehicle_type] * discount_factors[i]

    return df

df_with_time_based_toll_rates = calculate_time_based_toll_rates(df_unrolled_distance_with_toll)

# Print the resulting DataFrame
print(df_with_time_based_toll_rates[['index', 'id_end', 'start_day', 'start_time', 'end_day', 'end_time',  'moto', 'car', 'rv', 'bus', 'truck']])

        index   id_end start_day  \
0     1001400  1001402    Sunday   
1     1001400  1001404    Sunday   
2     1001400  1001406    Sunday   
3     1001400  1001408    Sunday   
4     1001400  1001410    Sunday   
...       ...      ...       ...   
1801  1004356  1001470    Sunday   
1802  1004356  1001472    Sunday   
1803  1004356  1001488    Sunday   
1804  1004356  1004354    Sunday   
1805  1004356  1004355    Sunday   

                                             start_time   end_day  \
0     [2023-01-01 00:00:00, 2023-01-01 10:00:00, 202...  Saturday   
1     [2023-01-01 00:00:00, 2023-01-01 10:00:00, 202...  Saturday   
2     [2023-01-01 00:00:00, 2023-01-01 10:00:00, 202...  Saturday   
3     [2023-01-01 00:00:00, 2023-01-01 10:00:00, 202...  Saturday   
4     [2023-01-01 00:00:00, 2023-01-01 10:00:00, 202...  Saturday   
...                                                 ...       ...   
1801  [2023-01-01 00:00:00, 2023-01-01 10:00:00, 202...  Saturday   
1802  [2023-01-